# NLP

In [26]:
import re
from string import punctuation, digits, ascii_lowercase

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import (train_test_split,cross_val_score,
                                      StratifiedShuffleSplit)
                                      
%matplotlib inline

In [3]:
reviews = pd.read_csv('yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.csv')

In [4]:
samp = reviews.sample(n=10000)

Define lists of escape sequences, digits, stopwords to use for parseing. Also defines the type of stemming to be used. 

In [5]:
escapes = ''.join([chr(char) for char in range(1, 32)])
removeables = escapes + digits 
stops = [str(word) for word in stopwords.words('english')] + list(ascii_lowercase)
sno = nltk.stem.SnowballStemmer('english')

Create function to process the text and then use it to make a new coloumn in the dataframe where all texted has been processed. 

In [6]:
def parse_text(text):
    ''' This function takes a review string and removes all escape sequences,
        digits, punctuation, http links, and stop words. Furthermore, every
        word in the string will be stemmed using nltk's snowball stemmer.
        Every word is also transformed to be lowercase.'''
    
    text = re.sub(r"http\S+", " ", text)
    regex = re.compile('[%s]' % re.escape(punctuation))
    text = regex.sub(' ', text)
    text = text.translate(None, removeables)
    text = text.decode('utf8')
    text = ' '.join([sno.stem(word.lower()) for word in text.split() if word.lower() not in set(stops)])
    return text

In [7]:
samp['parsed_text']=samp.text.apply(parse_text)

### Bag of words, tf-idf vectorization
Create the bag of words representation. Find counts of each word in document and in whole courpus. Then create the tfidf representation. Worth also considering binary count vecorizing, supposed to work better for smaller sample sets.

In [8]:
corpus = samp.parsed_text.tolist()

In [9]:
# count_vect = CountVectorizer()
# X_counts = count_vect.fit_transform(corpus)

In [10]:
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)


In [11]:
y = np.array(samp.stars.tolist())

In [12]:
y[:4]

array([5, 1, 5, 3])

### Train-Test split

Need to split the dataset into train and test, then use cross val to traing the classifier.

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.30,
                                                    stratify=y)

In [ ]:
sss = StratifiedShuffleSplit(y_train, 5, test_size=0.3, random_state=0)

### Training a classifier

In [14]:
mnb = MultinomialNB().fit(X_train,y_train)

In [15]:
preds = mnb.predict(X_test)

In [16]:
pd.Series(preds).hist()

In [17]:
pd.Series(y_test).hist()

### Accuracy Metrics

importnatn to consider what is the best accuracy measure to use to test results. notice that star rating is a ordered 